In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-customer-transaction-prediction-dataset/train.csv
/kaggle/input/santander-customer-transaction-prediction-dataset/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction-dataset/train.csv')

In [3]:
df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
df.shape

(200000, 202)

### Now we have to find and separate the variable columns

In [5]:
var_cols = [x for x in df.columns if x not in ['ID_code','target']]
len(var_cols)

200

##### Notice that the shape of var_cols (which stores all the input columns) has 2 less columns.

##### We now split the dataset into inputs and output

In [6]:
X = df.loc[:,var_cols]
y = df.iloc[:,1]

##### Next step is to split the data into training and validation set

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=13)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((160000, 200), (40000, 200), (160000,), (40000,))

##### Following action would be to fit the LightGBM model

In [8]:
import lightgbm

train_set = lightgbm.Dataset(X_train,label=y_train)
valid_set = lightgbm.Dataset(X_test,label=y_test)

##### Next course of action would be to specify parameters

In [9]:
params = {'objective':'binary',
          'metric':'auc',
          'is_unbalance':'true',
          'boosting':'gbdt',
          'num_leaves':63,
          'feature_fraction':0.5,
          'bagging_fraction':0.5,
          'bagging_freq':20,
          'learning_rate':0.01,
          'verbose':-1,
          'num_boost_round':1000,
          'early_stopping_rounds':10}

In [10]:
model = lightgbm.train(params,
                       train_set,
                       valid_sets=valid_set)

/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [11]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)


from sklearn.metrics import roc_auc_score
print("AUC Train: {:.4f}\nAUC Valid: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))

AUC Train: 0.9612
AUC Valid: 0.8888
